In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm

from astropy.cosmology import FlatLambdaCDM

In [2]:
time_frame = 365 + 366 + 365 + 365 + 365 + 366 + 365 + 365 + 365 + 366 + 365 + 365 + 365 + 366

In [3]:
def counting_old(number_sne, number_grb, falsehoods = 100, mode = None):
    
    total_prob = []
    
    for i in range(falsehoods):
        grbs = np.sort(np.random.rand(number_grb) * time_frame)
        sne = np.sort(np.random.rand(number_sne) * time_frame)
        diff_matrix = np.abs(grbs[:, None] - sne[None, :])
        
        within_tol = np.any(diff_matrix <= 0.15, axis=1)

        # Count how many grbs entries are within 0.3 of any sne value
        count = np.sum(within_tol)
        
        total_prob.append(count/(number_sne + number_grb))
    return np.nanmean(total_prob)

In [4]:
# def counting(number_sne, number_grb, trials=1000):
#     coincidence_counts = []

#     for _ in tqdm(range(trials), desc='Simulating coincidences'):
#         grbs = np.sort(np.random.rand(number_grb) * time_frame)
#         sne = np.sort(np.random.rand(number_sne) * time_frame)

#         count = 0
#         for sn in sne:
#             # Count if any GRB is within ± delta_t of this SN
#             if np.any(np.abs(grbs - sn) <= 0.3):
#                 count += 1
#         coincidence_counts.append(count)

#     return np.array(coincidence_counts)


In [5]:
def counting(number_sne, number_grb, falsehoods = 100):
    
    total_prob = []
    
    for i in tqdm(range(falsehoods), desc='Please'):
        
        temp_number_grb = np.random.poisson(number_grb/14) * 14
        temp_number_sne = np.random.poisson(number_sne/14) * 14
        
        grbs = np.sort(np.random.rand(temp_number_grb) * time_frame)
        sne = np.sort(np.random.rand(temp_number_sne) * time_frame)
        
        grb_counts = 0
        sne_counts = 0
        
        for grb in grbs:
            diff_matrix_count = np.nansum(np.abs(grb - sne) <= 0.3)
            if diff_matrix_count > 0:
                grb_counts += 1
        # print('GRB:', grb_counts)
        
        for sn in sne:
            diff_matrix_count = np.nansum(np.abs(grbs - sn) <= 0.3)
            
            if diff_matrix_count > 0:
                sne_counts += 1
                
        
        total_prob.append(( grb_counts + sne_counts - min([grb_counts, sne_counts]) ) / (temp_number_grb + temp_number_sne))

    # return ( grb_counts + sne_counts - min([grb_counts, sne_counts]) ) / (number_sne + number_grb)
    
    return np.nanmedian(total_prob), 

def symmetric_coincidence_probability(lambda_S, lambda_G, delta_t):
    """
    Symmetric probability that a randomly chosen SN or GRB has a counterpart
    within ±delta_t, under independent Poisson assumptions.

    Returns:
    - P_sym: weighted average probability over all events
    - p_SN_has_GRB: P(SN has GRB within ±delta_t)
    - p_GRB_has_SN: P(GRB has SN within ±delta_t)
    """
    p_SN_has_GRB = 1.0 - np.exp(-2.0 * lambda_G * delta_t)
    p_GRB_has_SN = 1.0 - np.exp(-2.0 * lambda_S * delta_t)

    P_sym = (lambda_S * p_SN_has_GRB + lambda_G * p_GRB_has_SN) / (lambda_S + lambda_G)
    return P_sym, p_SN_has_GRB, p_GRB_has_SN

In [6]:
file = '/Users/zgl12/Downloads/Summary_table (1).txt'

df  = pd.read_csv(file, delim_whitespace=True)

df

lengths = []

for i in range(11, 25):

    temp_df = df #df[df['GRB_name_Fermi'] != 'None']

    lengths.append(len(temp_df[temp_df['GRB_name'].str.startswith(f'GRB{i}')]))
    
grbs_year = np.nanmean(lengths)
grbs_year_err = np.nanstd(lengths, ddof = 1)

number_sne = int(487.62516272856794 * 14)

/var/folders/6v/pg_c9mqn3gn45gn_zmvy9w9cvfsx9j/T/ipykernel_28077/1688668002.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df  = pd.read_csv(file, delim_whitespace=True)


In [ ]:
xrb_fracs = np.linspace(0, 1, 101)

probs = []

for frac in tqdm(xrb_fracs, desc='Calculating Probabilities'):

    probs.append(counting_old(number_sne, int(np.nansum(lengths)*frac), falsehoods = 5000))
    
import scipy.stats as stats

probby = np.trapz(probs, xrb_fracs)
confidence_level = 1- probby * (1 - 0.99394678)
z_score = stats.norm.ppf((1 + confidence_level) / 2)
print(f"The z-score for a {probby*100:.3f}% confidence level is: {z_score:.3f}")

confidence_level = 1- probby #* (1 - 0.99394678)
z_score = stats.norm.ppf((1 + confidence_level) / 2)
print(f"The z-score for a {probby*100:.3f}% confidence level is: {z_score:.3f}")

Calculating Probabilities:  99%|█████████▉| 100/101 [14:42:53<17:06, 1026.76s/it]  

In [ ]:
counting(number_sne, int(np.nansum(lengths)/2), falsehoods = 5000)

In [ ]:
# ------------ EXAMPLE -------------
# rates (events / year)
lambda_SN  = 488  #   1 SN per 100 years
lambda_GRB = 2132 /14     #   3 GRBs per year
delta_t    = 0.3/365.25     #  ±0.01 years  (= ~3.65 days)

P_sym, p_SN, p_GRB = symmetric_coincidence_probability(lambda_SN, lambda_GRB, delta_t)
print(f"Probability Sym. in ±Δt : {P_sym:.5f}")
print(f"Probability SN has GRB in ±Δt : {p_SN:.5f}")
print(f"Probability GRB has SN in ±Δt : {p_GRB:.5f}")

In [ ]:
np.nanmedian(coincidences)/(number_sne + int(np.nansum(lengths)/2)), np.nanstd(coincidences, ddof = 1)#, np.nanmean(coincidences), np.nanpercentile(coincidences, [2.5, 97.5])

In [ ]:
import numpy as np
from tqdm import tqdm

time_frame = 365 + 366 + 365 + 365 + 365 + 366 + 365 + 365 + 365 + 366 + 365 + 365 + 365 + 366

def counting(number_sne, number_grb, falsehoods=100):
    total_unique_pairs = []

    for _ in tqdm(range(falsehoods), desc='Please'):
        temp_number_grb = np.random.poisson(number_grb)
        temp_number_sne = np.random.poisson(number_sne)
        
        grbs = np.sort(np.random.rand(temp_number_grb) * time_frame)
        sne = np.sort(np.random.rand(temp_number_sne) * time_frame)
        
        matched_grbs = set()
        matched_sne = set()
        unique_pairs = 0

        # For every GRB, try to find the nearest available SN within 0.3
        for i, grb in enumerate(grbs):
            for j, sn in enumerate(sne):
                if j in matched_sne:
                    continue
                if abs(grb - sn) <= 0.3:
                    matched_grbs.add(i)
                    matched_sne.add(j)
                    unique_pairs += 1
                    break  # Move to the next GRB after finding a match

        total_unique_pairs.append(unique_pairs/(temp_number_grb + temp_number_sne))
    
    return np.nanmedian(np.array(total_unique_pairs))

number_sne = int(487.62516272856794 * 14)
number_grb = 2132

counting(number_sne, number_grb, falsehoods=10)

In [ ]:
import numpy as np
from tqdm import tqdm

time_frame = 365 + 366 + 365 + 365 + 365 + 366 + 365 + 365 + 365 + 366 + 365 + 365 + 365 + 366  # days
# time_frame /= 365.25  # convert to years

def counting(number_sne, number_grb, falsehoods=1000, window=0.3):
    coincidence_fractions = []

    for _ in tqdm(range(falsehoods), desc='Simulating'):
        temp_number_grb = np.random.poisson(number_grb)
        temp_number_sne = np.random.poisson(number_sne)
        
        grbs = np.random.rand(temp_number_grb) * time_frame
        sne = np.random.rand(temp_number_sne) * time_frame
        
        matched_grbs = set()
        matched_sne = set()

        for i, grb in enumerate(grbs):
            for j, sn in enumerate(sne):
                if abs(grb - sn) <= window:
                    matched_grbs.add(i)
                    matched_sne.add(j)

        involved_events = len(matched_grbs) + len(matched_sne)
        total_events = temp_number_grb + temp_number_sne
        coincidence_fractions.append(involved_events / total_events if total_events > 0 else 0)

    return np.nanmedian(coincidence_fractions)

number_sne = int(487.62516272856794 * 14)
number_grb = 2132

counting(number_sne, number_grb, falsehoods=10, window=0.15)

In [ ]:
mu = 2 * 0.3/365.25 * number_sne /14

In [ ]:
1- np.exp(-mu)

In [ ]:
number_grb

In [ ]:
import numpy as np


